## Logisk imputering
<font size = 2>Logisk imputering er å tilordne en variabel en verdi utledet fra logiske regler:
- Hvis betingelse er sann, så skal verdi A settes: <code><b>when</b>(betingelse, A)</code>
- Hvis  betingelse er sann, så skal verdi A settes, hvis ikke, så skal verdi B settes: <code><b>when</b>(betingelse, A).<b>otherwise</b>(B)</code></font>

### Funksjoner for logisk imputering
<font size=2>Det er blitt laget tre funksjonene i Pyspark som hh.endrer en negative verdi til tallet 0, en negativ verdi til positiv og en missingverdi til null. Selve koden finnes i neste paragraf og eksempler på hvordan de brukes finnes lenger ned i noten:
<li> <code>set_neg_to_zero(df, col_names)</code>:
<li> <code>set_neg_to_plus(df, col_names)</code>:
<li> <code>set_miss_to_zero(df, col_names)</code>:
<ul>
</font>

In [ ]:
def set_neg_to_zero(df, col_names):
    for col_name in col_names:
            df = df.withColumn(col_name, F.when(df[col_name]<0, F.lit(0))\
            .otherwise(df[col_name]))
    return df
    
def set_neg_to_plus(df, col_names):
    for col_name in col_names:
            df = df.withColumn(col_name, F.when(df[col_name]<0, F.abs(df[col_name]))\
            .otherwise(df[col_name]))
    return df
    
def set_miss_to_zero(df, col_names):
    for col_name in col_names:
        df = df.withColumn(col_name, F.when(df[col_name].isNull(), F.lit(0))\
        .otherwise(df[col_name]))
    return df

#### Importere biblioteker (kode)

In [ ]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

### Hente eksempeldata (kode)

In [ ]:
df_areal = spark.read.path("/felles/veiledning/pyspark/eksempler/areal")
df_bnp = spark.read.path("/felles/veiledning/pyspark/eksempler/bnp")
df_innbyggerantall_2020 = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")
df_innbyggerantall_2018 = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2018")

#### Koble innbyggerantall på arealdatasett (koblingsnøkkel er Landkode) til bruk for å illustrere logisk imputering (kode)

In [ ]:
df_areal_lite = df_areal.select('Landkode','Areal').withColumnRenamed('Landkode','Lkode')
df_areal_innbyggerantall_20 = df_areal_lite.join(df_innbyggerantall_2020.select('Landkode','Innbyggerantall'), df_areal_lite.Lkode == df_innbyggerantall_2020.Landkode, how='left')
df_areal_innbyggerantall_20.show()

#### Erstatte missing-verdier i innbyggertall med null ved bruk av logisk betingelse (kode)

In [ ]:
df_areal_innbyggerantall_imp = df_areal_innbyggerantall_20.withColumn('Innbyggere', F.when(df_areal_innbyggerantall_20['Innbyggerantall'].isNull(),F.lit(0)).otherwise(df_areal_innbyggerantall_20['Innbyggerantall']))
df_areal_innbyggerantall_imp.filter(df_areal_innbyggerantall_imp.Innbyggerantall.isNull()).show()

#### Erstatte missing-verdier i innbyggertall ved bruk av funksjonen set_miss_to_zero (kode)

In [ ]:
df_areal_innbyggerantall_imp2 = set_miss_to_zero(df_areal_innbyggerantall_20, ['Innbyggerantall'])
df_areal_innbyggerantall_imp2.filter(df_areal_innbyggerantall_imp2.Lkode.isin(['RO','GL','CZ','SK'])).show()

#### Erstatte negativ verdi i bnp med 0 ved bruk av logisk regel (kode)

In [ ]:
df_bnp_imp = df_bnp.withColumn('BNPimp', F.when(df_bnp['BNP']<0,F.lit(0)).otherwise(df_bnp['BNP']))
df_bnp_imp.filter(df_bnp_imp['BNP'] < 0).show()

#### Erstatte negativ verdi i bnp med 0 ved bruk av funksjonen set_neg_to_zero (kode)

In [ ]:
df_bnp_imp2 = set_neg_to_zero(df_bnp,['BNP'])
df_bnp_imp2.filter(df_bnp_imp2.Landkode.isin(['PT','SI'])).show()

#### Erstatte negativ verdi av bnp med absoluttverdien ved bruk av logisk regel (kode)

In [ ]:
df_bnp_imp_abs = df_bnp.withColumn('BNPpos',F.when(df_bnp['BNP']<0, F.abs(df_bnp['BNP']))\
            .otherwise(df_bnp['BNP']))
df_bnp_imp_abs.filter(df_bnp_imp2.Landkode.isin(['PT','SI'])).show()

#### Erstatte negativ verdi med absoluttverdien ved bruk av funksjonen set_neg_to_plus (kode)

In [ ]:
df_bnp_imp_abs2 = set_neg_to_plus(df_bnp,['BNP'])
df_bnp_imp_abs2.filter(df_bnp_imp2.Landkode.isin(['PT','SI'])).show()

#### Lage datasett til bruk i eksempel på historisk imputering.
Først lages et mindre datasett over innbyggerantall i fire land i 2020. Deretter kobles et datsett med tall fra 2020 og et med tall fra 2018 sammen i neste paragraf. Variablene på datasettet fra 2018 gis nye navn.

In [ ]:
df_innbyggerantall_2020_lite = df_innbyggerantall_2020.select(df_innbyggerantall_2020.columns).filter(df_innbyggerantall_2020.Landkode.isin(['FO','SE','BE','DK']))
df_innbyggerantall_2020_lite.show()

In [ ]:
df_innbyggere_2018 = df_innbyggerantall_2018.withColumnRenamed('Innbyggerantall','Innbyggerantall_2018').withColumnRenamed('År','Årstall').withColumnRenamed('Kilde','Kilde_2018').withColumnRenamed('Landkode','Landkode_2018')
df_innbyggere_2020_2018 = df_innbyggerantall_2020_lite.join(df_innbyggere_2018, 'Land', 'left')
df_innbyggere_2020_2018.show()

#### Til slutt lages et nytt datasett der verdier fra variablene fra 2018-datasettet er satt inn for verdiene der innbyggerantallet var 0.
Samtidig settes årstallet lik 2018 der vi har imputert 2018-tall og kilden endres til kilden fra 2018.

In [ ]:
df_midl = df_innbyggere_2020_2018.withColumn('Innbyggere', F.when(df_innbyggere_2020_2018['Innbyggerantall'] <= 0, df_innbyggere_2020_2018['Innbyggerantall_2018']).otherwise(df_innbyggere_2020_2018['Innbyggerantall']))\
                                 .withColumn('Aar', F.when(df_innbyggere_2020_2018['Innbyggerantall'] <= 0, df_innbyggere_2020_2018['Årstall']).otherwise(df_innbyggere_2020_2018['År'])) \
                                 .withColumn('Kilde_ny', F.when(df_innbyggere_2020_2018['Innbyggerantall'] <= 0, df_innbyggere_2020_2018['Kilde_2018']).otherwise(df_innbyggere_2020_2018['Kilde']))
    
df_innbyggere_imp = df_midl.select(['Land','Innbyggere','Aar','Kilde_ny']).withColumnRenamed('Innbyggere','Innbyggerantall').withColumnRenamed('Aar','År').withColumnRenamed('Kilde_ny','Kilde')
df_innbyggere_imp.show(10,False)